In [14]:
import os

In [39]:
pip install langchain langchain_openai wikipedia-api

In [40]:
pip install -U langchain-community

In [6]:
os.environ["AZURE_OPENAI_API_KEY"] = "DTN1wiFDfM0FczleBcGBCVkOy7Clf6mEsRDf7KRDXNDcqUItGTKqJQQJ99ALACHYHv6XJ3w3AAAAACOGIxqq"
os.environ["AZURE_OPENAI_DEPLOYMENT"]='gpt-35-turbo-16k'
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-10-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"]='https://delvi-m4wlt8x5-eastus2.cognitiveservices.azure.com/'

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [7]:
model.invoke("Hello, world!").content

'Hello! How can I assist you today?'

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [9]:
summary_prompt = PromptTemplate(
    input_variables = ["text"],
    template = "Summarize the following text: {text}"
)

title_prompt = PromptTemplate(
    input_variables = ["summary"],
    template = "Create a title for the following summary: {summary}"
)


In [10]:
summary_chain = LLMChain(llm=model, prompt=summary_prompt)
title_chain = LLMChain(llm=model, prompt=title_prompt)

<ipython-input-10-22f62cad76b8>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=model, prompt=summary_prompt)


In [11]:
overall_chain = SimpleSequentialChain(chains=[summary_chain, title_chain])

In [12]:
input_text = "Artificial intelligence is a branch of computer science that aims to create machines that can perform tasks that typically require human intelligence. These tasks include problem-solving, learning, and pattern recognition."
result = overall_chain.run(input_text)


<ipython-input-12-955f6fa404f1>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = overall_chain.run(input_text)


In [13]:
result

'"The Rise of Artificial Intelligence: Unlocking the Potential of Human-like Machines"'

In [47]:
pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 930.8 kB/s eta 0:00:00


In [50]:
from langchain.document_loaders import WikipediaLoader
import wikipedia
from langchain.agents import initialize_agent, Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wiki_tool = Tool(
    name="Wikipedia Extractor",
    func=wikipedia,
    description="Fetches summaries from Wikipedia for a given query."
)

python_repl = PythonREPL()

repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

tools = [wiki_tool, repl_tool]
agent = initialize_agent(tools, model, agent="zero-shot-react-description", verbose=True)

In [51]:
query = "Tell me about Python programming language and calculate 2 + 2."
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
I can use the Wikipedia Extractor tool to fetch information about Python programming language, and then use the python_repl tool to calculate 2 + 2.
Action: Wikipedia Extractor
Action Input: 'Python programming language'
Observation: Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revisi

I have obtained the information about the Python programming language from Wikipedia. Now, I can use the python_repl tool to calculate 2 + 2.
Action: python_repl
Action Input: print(2 + 2)
Observation: 4

Thought:I have obtained the result of the calculation 2 + 2 using the python_repl tool.
Final Answer: The Python programming language is a high-level, general-purpose programming language known for its code readability and comprehensive standard library. It supports multiple programming paradigms and is widely used in the machine learning community. Additionally, the result of the calculation 2 + 2 is 4.

> Finished chain.
The Python programming language is a high-level, general-purpose programming language known for its code readability and comprehensive standard library. It supports multiple programming paradigms and is widely used in the machine learning community. Additionally, the result of the calculation 2 + 2 is 4.
